<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: KNN #


##Inicialización##

In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

##Pre procesamiento##

In [15]:
#muy suceptible al ruido
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 71 (delta 33), reused 37 (delta 14), pack-reused 0
Unpacking objects: 100% (71/71), done.
/content/fiumark/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


Para KNN es importante escalar los datos. 

In [16]:
pd.options.display.max_columns = None

df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)


##Entrenamiento##

In [17]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_neighbors': np.arange(1, 100),
    'weights': ['uniform', 'distance'],
    'metric': ['cosine','minkowski'],
    'algorithm':['ball_tree','kd_tree','brute']
}

clf = KNeighborsClassifier()

rgscv = RandomizedSearchCV(
    clf, params, n_iter=200, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [18]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8102941176470587
Best params {'weights': 'uniform', 'n_neighbors': 20, 'metric': 'minkowski', 'algorithm': 'brute'}


##Evaluación del modelo##

In [19]:
from sklearn.metrics import classification_report

print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.77      0.97      0.86        70
         1.0       0.94      0.61      0.74        51

    accuracy                           0.82       121
   macro avg       0.86      0.79      0.80       121
weighted avg       0.84      0.82      0.81       121



##Predicción##

In [21]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
df2 = pd.DataFrame(rgscv.predict(X),columns=['volveria'])
df3 = pd.concat([X_holdout.id_usuario,df2],axis=1)
df3.to_csv('knn.csv')
df3

,id_usuario,volveria
0,650,0.0
1,234,0.0
2,31,0.0
3,552,0.0
4,411,0.0
...,...,...
85,354,0.0
86,82,0.0
87,172,0.0
88,8,0.0
